# Exploring `PyPestUtils` with a structured Freyberg model

In [ ]:
import os
import sys
import shutil
import subprocess as sp

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

Setup the model dir

In [ ]:
org_d = "freyberg_daily"
w_d = "freyberg"
if os.path.exists(w_d):
    shutil.rmtree(w_d)
shutil.copytree(org_d,w_d)

Eventually will add some post processing stuffs but for now no need to run the model...

In [ ]:
#p = sp.Popen(["mf6"],cwd=w_d)

In [ ]:
delc = np.loadtxt(os.path.join(w_d,"freyberg6.dis_delc.txt")).flatten()
delr = np.loadtxt(os.path.join(w_d,"freyberg6.dis_delr.txt")).flatten()

In [ ]:
nrow = delc.shape[0]
ncol = delr.shape[0]
ib = np.loadtxt(os.path.join(w_d,"freyberg6.dis_idomain_layer1.txt"),dtype=int)
ib = ib.flatten().reshape(nrow,ncol)
plt.imshow(ib)

## Some parameterization stuffs
Point to the pypestutils source dir and import

In [ ]:
sys.path.insert(0,"..")
from pypestutils.pestutilslib import PestUtilsLib
lib = PestUtilsLib()

Setup some dimensions for the grid

In [ ]:

evec = np.arange(ncol)
nvec = np.arange(nrow)
easting,northing = np.meshgrid(evec,nvec)

In [ ]:
easting.shape,northing.shape

In [ ]:
easting

In [ ]:
easting = easting.flatten()
northing = northing.flatten()

In [ ]:
easting.shape,northing.shape

In [ ]:
# cell area
area = np.ones_like(easting)
#active array
active = ib.flatten().astype(int)#np.ones_like(easting,dtype=int)
# property mean
mean = np.ones_like(easting)
# property variance
var = np.ones_like(easting)
# the variogram range
aa = np.ones_like(easting)*5
# anisotropy
anis = np.ones_like(easting)*5
# bearing
bearing = (np.ones_like(easting) * 45)

First initial the random engine with seed

In [ ]:
lib.initialize_randgen(12345)

In [ ]:
# generate some reals

In [ ]:
transform = "none"
variogram_type = "exp"
power = 1.0 #unused
num_reals = 10
reals = lib.fieldgen2d_sva(easting,northing,area,active,mean,var,aa,anis,bearing,transform,variogram_type,power,num_reals)
print(reals)

In [ ]:
plt.imshow(reals[:,0].reshape((nrow,ncol)))

Do something weird with bearing: make it a function of easting?

In [ ]:
bearing = np.add(np.ones((nrow,ncol)),np.atleast_2d(np.arange(ncol)))

In [ ]:
cb = plt.imshow(bearing)
plt.colorbar(cb)

In [ ]:
bearing = bearing.flatten()

In [ ]:
anis *= 2
aa *= 2

In [ ]:
reals = lib.fieldgen2d_sva(easting,northing,area,active,mean,var,aa,anis,bearing,transform,variogram_type,power,num_reals)
r = reals[:,0].reshape((nrow,ncol))
plt.imshow(r)

Now some pilot points with spatially varying variogram props.  Just sample the realization for pp values

In [ ]:
ppeasting,ppnorthing = [],[]
ppval = []
easting,northing = np.meshgrid(evec,nvec)
pp_space = 20
half_pp_space = int(pp_space/2)
for i in range(half_pp_space,nrow,pp_space):
    for j in range(half_pp_space,ncol,pp_space):
        if ib[i,j] == 0:
            continue
        ppeasting.append(easting[i,j])
        ppnorthing.append(northing[i,j])
        ppval.append(r[i,j])
ppeasting = np.array(ppeasting)
ppnorthing = np.array(ppnorthing)
ppval = np.array(ppval)
ppeasting.shape,ppnorthing.shape,ppval.shape

In [ ]:
fig,axes = plt.subplots(1,2)
ax = axes[0]
ax.imshow(r)
ax.set_title("realization")
ax = axes[1]
ax.set_aspect("equal")
ax.set_title("pp locs with sampled values")
ax.set_ylim(0,nrow)
ax.set_xlim(0,ncol)
ax.scatter(ppeasting,ppnorthing,marker=".",s=50,c=ppval)
ax.invert_yaxis()

Now calculate kriging factors

In [ ]:
easting = easting.flatten()
northing = northing.flatten()
max_pts = 50
min_pts = 1
search_dist = 1.e+10
aa_pp = aa * pp_space *10 #?
zone_pp = np.ones_like(ppeasting,dtype=int)
fac_file = os.path.join(w_d,"factors.bin")
from datetime import datetime
s = datetime.now()
ipts = lib.calc_kriging_factors_2d(ppeasting,ppnorthing,zone_pp,easting,northing,ib.flatten(),
                                   "exp","ordinary",aa_pp,anis,bearing,search_dist,max_pts,min_pts,fac_file,"binary")
"total points:",ipts," took:",(datetime.now() - s).total_seconds()

Interpolate to the grid.  I think if we are estimating changing the variogram properties, we have to call `calc_kriging_factors()` each time...

In [ ]:
result = lib.krige_using_file(fac_file,"binary",len(easting),"ordinary","none",np.array(ppval),np.zeros(len(ppval)))

In [ ]:
rr = result["targval"].reshape(nrow,ncol)
fig,axes = plt.subplots(1,2)
ax = axes[0]
ax.set_title("pp interpolated array")
ax.imshow(rr) #the interpolated array
ax = axes[1]
ax.set_aspect("equal")
ax.set_title("pp locs with sampled values")
ax.set_ylim(0,nrow)
ax.set_xlim(0,ncol)
ax.scatter(ppeasting,ppnorthing,marker=".",s=50,c=ppval)
ax.invert_yaxis()

Now flip the bearing just for fun...

In [ ]:
bearing_inv = bearing * -1
fac_file = os.path.join(w_d,"factors.txt")
ipts = lib.calc_kriging_factors_2d(ppeasting,ppnorthing,zone_pp,easting,northing,ib.flatten(),
                                   "exp","ordinary",aa_pp,anis,bearing_inv,search_dist,max_pts,min_pts,fac_file,"text")
result = lib.krige_using_file(fac_file,"text",len(easting),"ordinary","none",np.array(ppval),np.zeros(len(ppval)))
rr = result["targval"].reshape(nrow,ncol)
fig,axes = plt.subplots(1,2)
ax = axes[0]
ax.set_title("pp interpolated array")
ax.imshow(rr) #the interpolated array
ax = axes[1]
ax.set_title("pp locs with sampled values")
ax.set_ylim(0,nrow)
ax.set_xlim(0,ncol)
ax.scatter(ppeasting,ppnorthing,marker=".",s=50,c=ppval)
ax.invert_yaxis()